In [5]:
import pandas as pd
from utils.audio_tools import extract_clip_and_mfcc, extract_negative_clip_and_mfcc
import os

## Step 1: Load Annotations

Here we read the TSV file containing orca call timings and preview the data.


In [6]:
annotations = pd.read_csv("annotations.tsv", sep="\t")

In [7]:
annotations.head()

,dataset,wav_filename,start_time_s,duration_s,location,date,pst_or_master_tape_identifier
0,podcast_round1,60012.wav,34.126,2.918,"Dabob Bay, Seattle, Washington",1960-10-28,60012
1,podcast_round1,60012.wav,36.816,2.588,"Dabob Bay, Seattle, Washington",1960-10-28,60012
2,podcast_round1,60012.wav,42.550,2.055,"Dabob Bay, Seattle, Washington",1960-10-28,60012
3,podcast_round1,60012.wav,44.606,2.410,"Dabob Bay, Seattle, Washington",1960-10-28,60012
4,podcast_round1,60012.wav,46.636,3.425,"Dabob Bay, Seattle, Washington",1960-10-28,60012


## Step 2: Extract Clips

Here we take a wav file, clip the call, break it into small fragments, and then average the mfcc of these fragments while writing the clip to a file.

We're looping through a few rows of the annotations file. For each row:

We get the path to the source WAV file.

We name the new clip file based on its position in the loop.

We define where the new clip will be saved (in clips/positive/).

We run extract_clip_and_mfcc(), which:

Cuts and saves the audio segment

Returns a 13-value MFCC vector

We build a new record (dictionary) with the clip name, label, and MFCC values

We add that record to a list

After looping, we turn the list into a DataFrame and save it as mfcc_features.csv.


In [8]:
import shutil

# Delete entire clips folder
shutil.rmtree("data/clips", ignore_errors=True)

# Recreate the folder structure
os.makedirs("data/clips/positive", exist_ok=True)
os.makedirs("data/clips/negative", exist_ok=True)

print("Clips folder reset.")

records = []
pos_count = 0
neg_count = 0
i = 0

while pos_count < 100 or neg_count < 100:
    if i >= len(annotations):
        print("Reached end of annotations.")
        break

    row = annotations.iloc[i]
    wav_path = os.path.join("data", "wav", row["wav_filename"])

    # POSITIVE
    if pos_count < 100:
        pos_clip_name = f"clip_{pos_count+1:05}.wav"
        pos_output_path = os.path.join("data", "clips", "positive", pos_clip_name)

        mfcc_pos = extract_clip_and_mfcc(
            wav_path=wav_path,
            start_time=row["start_time_s"],
            duration=row["duration_s"],
            output_path=pos_output_path
        )

        if mfcc_pos is not None:
            record = {
                "clip_name": pos_clip_name,
                "label": "orca_call",
                "source_wav": row["wav_filename"],
                "start_time": row["start_time_s"]
            }
            
            for j, val in enumerate(mfcc_mean):       # each MFCC value
                record[f"mfcc_{j+1}"] = val
            
            for j, val in enumerate(delta_mean):      # each delta value
                if j < 10:  # only keep the top 10 deltas
                    record[f"delta_mfcc_{j+1}"] = val
            
            records.append(record)
            pos_count += 1

    # NEGATIVE
    if neg_count < 100:
        same_file_rows = annotations[annotations["wav_filename"] == row["wav_filename"]]
        buffer = 1.0  # 1 second before and after each call
        exclude_ranges = [
            (max(0, r["start_time_s"] - buffer), r["start_time_s"] + r["duration_s"] + buffer)
            for _, r in same_file_rows.iterrows()
        ]


        neg_clip_name = f"neg_{neg_count+1:05}.wav"
        neg_output_path = os.path.join("data", "clips", "negative", neg_clip_name)

        mfcc_neg = extract_negative_clip_and_mfcc(
            wav_path=wav_path,
            exclude_ranges=exclude_ranges,
            output_path=neg_output_path
        )

        if mfcc_neg is not None:
            record = {"clip_name": neg_clip_name, "label": "no_call"}
            for j, val in enumerate(mfcc_neg):
                record[f"mfcc_{j+1}"] = val
            records.append(record)
            neg_count += 1

    i += 1

# Save the dataset
df = pd.DataFrame(records)
df.to_csv("mfcc_features.csv", index=False)
df["label"].value_counts()
df.head()

✅ Clips folder reset.


,clip_name,label,mfcc_1,mfcc_2
0,clip_00001.wav,orca_call,"[-376.07745290662814, 145.99649923670117, 23.4...","[-0.02086184866935607, 0.0448712116634677, -0...."
1,neg_00001.wav,no_call,"[-301.0517609887432, 124.3663819229527, 24.352...","[0.10243396141942442, -0.045156569134000055, -..."
2,clip_00002.wav,orca_call,"[-397.26383612881983, 147.1446965578204, 29.37...","[-0.16113247494216532, 0.08628232955281459, -0..."
3,neg_00002.wav,no_call,"[-310.4162239188598, 128.30735895607936, 23.81...","[0.22200175168946956, -0.08750420513051796, 0...."
4,clip_00003.wav,orca_call,"[-325.2725974569617, 143.61668120212312, 11.43...","[0.7359149004823944, -0.3135776147893503, 0.25..."


## Step 3: Train a Logistic Regression Baseline

We train a simple logistic regression model using the averaged MFCC features to classify clips as either orca calls or ambient noise.


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Load features
df = pd.read_csv("mfcc_features.csv")

# Split features (X) and labels (y)
# Input Data (mfcc's)
X = df[[f"mfcc_{i+1}" for i in range(13)]]
# Output Data (call or no call)
y = df["label"]

# Encode labels (orca_call → 1, no_call → 0)
y = y.map({"orca_call": 1, "no_call": 0})

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["no_call", "orca_call"]))


KeyError: "['mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13'] not in index"

## Step 4: Visualize Feature Importance

We plot the logistic regression model's learned coefficients to see which MFCC values (features) have the biggest influence on its predictions.


In [ ]:
import matplotlib.pyplot as plt

# Get feature importance (model coefficients)
coefficients = model.coef_[0]  # shape: (13,)

# Create a bar chart
plt.figure(figsize=(10, 4))
plt.bar(range(1, 14), coefficients)
plt.xticks(range(1, 14), [f"MFCC {i}" for i in range(1, 14)])
plt.axhline(0, color='gray', linestyle='--')
plt.xlabel("MFCC Coefficient")
plt.ylabel("Model Weight")
plt.title("MFCC Feature Importance (Logistic Regression)")
plt.tight_layout()
plt.show()


## Step 5: Train a Random Forest Classifier

We train a Random Forest model using the same training and test data to compare its performance to the logistic regression baseline.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Train the random forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_rf = rf_model.predict(X_test)

print("Confusion Matrix (Random Forest):")
print(confusion_matrix(y_test, y_pred_rf))

print("\nClassification Report (Random Forest):")
print(classification_report(y_test, y_pred_rf, target_names=["no_call", "orca_call"]))


## Step 6: Feature Importance (Random Forest)

We visualize which MFCC features the Random Forest model uses most to classify clips as orca or non-orca.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get feature importances from the trained random forest
importances = rf_model.feature_importances_

# Feature labels
feature_names = [f"MFCC {i+1}" for i in range(13)]

# Plot
plt.figure(figsize=(10, 4))
plt.bar(range(1, 14), importances)
plt.xticks(ticks=range(1, 14), labels=feature_names)
plt.ylabel("Importance")
plt.title("Feature Importance (Random Forest)")
plt.tight_layout()
plt.show()


## Step 7: Standardize MFCC Features

We scale all MFCC features to have mean 0 and standard deviation 1 using z-score standardization. This helps models treat all features equally, especially those sensitive to feature scale.


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Load dataset
df = pd.read_csv("mfcc_features.csv")

# Split features and labels
X = df[[f"mfcc_{i+1}" for i in range(13)]]
y = df["label"].map({"orca_call": 1, "no_call": 0})

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the MFCC features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression on scaled features
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_scaled)
print("Confusion Matrix (Logistic Regression, Scaled):")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report (Logistic Regression, Scaled):")
print(classification_report(y_test, y_pred, target_names=["no_call", "orca_call"]))


## Step 8: Extract Clips with MFCC + Delta Features

We re-extract 100 positive and 100 negative clips, this time saving both MFCC means and MFCC deltas. This adds temporal information to each sample and expands the feature set from 13 to 26.


In [ ]:
from utils.audio_tools import extract_clip_and_mfcc, extract_negative_clip_and_mfcc
import pandas as pd
import os

records = []
pos_count = 0
neg_count = 0
i = 0

while pos_count < 100 or neg_count < 100:
    if i >= len(annotations):
        print("Reached end of annotations.")
        break

    row = annotations.iloc[i]
    wav_path = os.path.join("data", "wav", row["wav_filename"])

    # === POSITIVE CLIP ===
    if pos_count < 100:
        pos_clip_name = f"clip_{pos_count+1:03}.wav"
        pos_output_path = os.path.join("data", "clips", "positive", pos_clip_name)

        mfcc_mean, delta_mean = extract_clip_and_mfcc(
            wav_path=wav_path,
            start_time=row["start_time_s"],
            duration=row["duration_s"],
            output_path=pos_output_path
        )

        if mfcc_mean is not None:
            record = {
                "clip_name": pos_clip_name,
                "label": "orca_call"
            }
            for j, val in enumerate(mfcc_mean):
                record[f"mfcc_{j+1}"] = val
            for j, val in enumerate(delta_mean):
                record[f"delta_mfcc_{j+1}"] = val
            records.append(record)
            pos_count += 1

    # === NEGATIVE CLIP ===
    if neg_count < 100:
        same_file_rows = annotations[annotations["wav_filename"] == row["wav_filename"]]
        exclude_ranges = [
            (r["start_time_s"], r["start_time_s"] + r["duration_s"])
            for _, r in same_file_rows.iterrows()
        ]

        neg_clip_name = f"neg_{neg_count+1:03}.wav"
        neg_output_path = os.path.join("data", "clips", "negative", neg_clip_name)

        mfcc_mean, delta_mean = extract_negative_clip_and_mfcc(
            wav_path=wav_path,
            exclude_ranges=exclude_ranges,
            output_path=neg_output_path
        )

        if mfcc_mean is not None:
            record = {
                "clip_name": neg_clip_name,
                "label": "no_call"
            }
            for j, val in enumerate(mfcc_mean):
                record[f"mfcc_{j+1}"] = val
            for j, val in enumerate(delta_mean):
                record[f"delta_mfcc_{j+1}"] = val
            records.append(record)
            neg_count += 1

    i += 1

# Save to new dataset
df = pd.DataFrame(records)
df.to_csv("mfcc_deltas_features.csv", index=False)
df["label"].value_counts()


## Step 9: Inspect MFCC + Delta Feature Distributions

We visualize a few MFCC and delta MFCC features to observe how they vary between orca calls and ambient noise.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load full dataset
df = pd.read_csv("mfcc_deltas_features.csv")

# Pick a few features to visualize
features_to_plot = ["mfcc_3", "mfcc_5", "delta_mfcc_2", "delta_mfcc_6"]

# Set up plots
plt.figure(figsize=(12, 8))
for i, feature in enumerate(features_to_plot):
    plt.subplot(2, 2, i + 1)
    sns.histplot(data=df, x=feature, hue="label", kde=True, element="step", common_norm=False)
    plt.title(f"Distribution of {feature}")
    plt.xlabel(feature)
    plt.ylabel("Count")
plt.tight_layout()
plt.show()


## Step 10: Rank Features by Class Separation
Mean difference between orca_call and no_call for each feature


In [ ]:
import numpy as np

# Load your dataset
df = pd.read_csv("mfcc_deltas_features.csv")

# Split by class
orca = df[df["label"] == "orca_call"]
no_call = df[df["label"] == "no_call"]

# Get all feature columns (exclude clip_name and label)
feature_cols = [col for col in df.columns if col.startswith("mfcc")]

# Calculate absolute mean differences
mean_diffs = {
    feature: abs(orca[feature].mean() - no_call[feature].mean())
    for feature in feature_cols
}

# Sort by largest difference
sorted_features = sorted(mean_diffs.items(), key=lambda x: x[1], reverse=True)

# Display results
print("Top features ranked by mean difference between classes:\n")
for i, (feat, diff) in enumerate(sorted_features[:10], 1):
    print(f"{i:>2}. {feat:<15} | Mean Difference: {diff:.4f}")


## Step 11: Visualize Delta MFCC Feature Distributions

We now inspect the delta MFCCs, which represent how each MFCC changes over time. These features often capture transitions and modulation, which can be very useful in detecting orca calls.


In [ ]:
# Load updated dataset
df = pd.read_csv("mfcc_deltas_features.csv")

# Choose some delta MFCCs to visualize
delta_features_to_plot = ["delta_mfcc_2", "delta_mfcc_3", "delta_mfcc_6", "delta_mfcc_9"]

# Set up plots
plt.figure(figsize=(12, 8))
for i, feature in enumerate(delta_features_to_plot):
    plt.subplot(2, 2, i + 1)
    sns.histplot(data=df, x=feature, hue="label", kde=True, element="step", common_norm=False)
    plt.title(f"Distribution of {feature}")
    plt.xlabel(feature)
    plt.ylabel("Count")
plt.tight_layout()
plt.show()

# Load dataset
df = pd.read_csv("mfcc_deltas_features.csv")

# Split by class
orca = df[df["label"] == "orca_call"]
no_call = df[df["label"] == "no_call"]

# Get delta feature columns
delta_features = [col for col in df.columns if col.startswith("delta_mfcc")]

# Compute absolute mean differences
delta_diffs = {
    feature: abs(orca[feature].mean() - no_call[feature].mean())
    for feature in delta_features
}

# Sort and display
sorted_deltas = sorted(delta_diffs.items(), key=lambda x: x[1], reverse=True)

print("Delta MFCCs ranked by class separation:\n")
for i, (feat, diff) in enumerate(sorted_deltas, 1):
    print(f"{i:>2}. {feat:<17} | Mean Difference: {diff:.4f}")


## Step 12: Compare Random Forest Performance (Full vs. Cleaned Features)

We train and evaluate two Random Forest models:
1. Using all 26 features (MFCCs + delta MFCCs)
2. Using a reduced set without the 3 weakest delta features

This comparison helps us see whether trimming weak features benefits a non-linear model like Random Forest.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load full dataset
df = pd.read_csv("mfcc_deltas_features.csv")
y = df["label"].map({"orca_call": 1, "no_call": 0})

# Define feature sets
full_features = [col for col in df.columns if col.startswith("mfcc") or col.startswith("delta_mfcc")]
cleaned_features = [f for f in full_features if f not in ["delta_mfcc_11", "delta_mfcc_12", "delta_mfcc_13"]]

# Split and train both models
for label, features in [("Full Features", full_features), ("Cleaned Features", cleaned_features)]:
    print(f"\n {label}:\n" + "-"*40)

    X = df[features]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=["no_call", "orca_call"]))


## Step 13: Train on All Available Clips

Loop through every row in annotations.tsv (not just 100)

Extract positive + negative clips (like before)

Compute MFCC + delta features (with weak deltas removed)

Save to a new dataset (e.g. mfcc_deltas_full.csv)

In [8]:
from utils.audio_tools import extract_clip_and_mfcc, extract_negative_clip_and_mfcc
import pandas as pd
import os

records = []
pos_count = 0
neg_count = 0

print("Starting full extraction...")

for i, row in annotations.iterrows():
    wav_path = os.path.join("data", "wav", row["wav_filename"])

    # === POSITIVE CLIP ===
    pos_clip_name = f"clip_{pos_count+1:05}.wav"
    pos_output_path = os.path.join("data", "clips", "positive", pos_clip_name)

    mfcc_mean, delta_mean = extract_clip_and_mfcc(
        wav_path=wav_path,
        start_time=row["start_time_s"],
        duration=row["duration_s"],
        output_path=pos_output_path
    )

    if mfcc_mean is not None:
        record = {
            "clip_name": pos_clip_name,
            "label": "orca_call"
        }
        for j, val in enumerate(mfcc_mean):
            record[f"mfcc_{j+1}"] = val
        for j, val in enumerate(delta_mean):
            if j < 10:  # only keep delta_mfcc_1 through delta_mfcc_10
                record[f"delta_mfcc_{j+1}"] = val
        records.append(record)
        pos_count += 1

    # === NEGATIVE CLIP ===
    same_file_rows = annotations[annotations["wav_filename"] == row["wav_filename"]]
    exclude_ranges = [
        (r["start_time_s"], r["start_time_s"] + r["duration_s"])
        for _, r in same_file_rows.iterrows()
    ]

    neg_clip_name = f"neg_{neg_count+1:05}.wav"
    neg_output_path = os.path.join("data", "clips", "negative", neg_clip_name)

    mfcc_neg, delta_neg = extract_negative_clip_and_mfcc(
        wav_path=wav_path,
        exclude_ranges=exclude_ranges,
        output_path=neg_output_path
    )

    if mfcc_neg is not None:
        record = {
            "clip_name": neg_clip_name,
            "label": "no_call"
        }
        for j, val in enumerate(mfcc_neg):
            record[f"mfcc_{j+1}"] = val
        for j, val in enumerate(delta_neg):
            if j < 10:
                record[f"delta_mfcc_{j+1}"] = val
        records.append(record)
        neg_count += 1

    if i % 500 == 0:
        print(f"Processed {i}/{len(annotations)} rows...")

# Save to CSV
df = pd.DataFrame(records)
df.to_csv("mfcc_deltas_full.csv", index=False)

print(f"\n✅ Extraction complete. Positives: {pos_count}, Negatives: {neg_count}")


Starting full extraction...
Processed 0/5537 rows...


C:\Users\joshu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Error extracting clip: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Processed 500/5537 rows...
Processed 1000/5537 rows...
Skipping: clip too short or empty
Error extracting clip: when mode='interp', width=9 cannot exceed data.shape[axis]=8
Skipping: clip too short or empty
Skipping: clip too short or empty
Error extracting clip: when mode='interp', width=9 cannot exceed data.shape[axis]=5
Processed 1500/5537 rows...
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Processed 2000/5537 rows...
Processed 2500/5537 rows...
Processed 3000/5537 rows...
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Skipping: clip too short or empty
Ski

C:\Users\joshu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=2
  warnings.warn(


Error extracting negative clip: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Skipping: clip too short or empty
Error extracting negative clip: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Skipping: clip too short or empty
Error extracting negative clip: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Skipping: clip too short or empty
Error extracting negative clip: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Skipping: clip too short or empty
Error extracting negative clip: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Skipping: clip too short or empty
Error extracting negative clip: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Skipping: clip too short or empty
Error extracting negative clip: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Skipping: clip too short or empty
Error extracting negative clip: when mode='interp', width=9 cannot exceed data.shape[axis]=1
Skipping: clip too

## Step 14: Train Final Random Forest on Full Dataset

We now train our tuned Random Forest model on all 4,533 positive and 2,411 negative clips. The feature set includes 13 MFCC means and 10 delta MFCC means, with the 3 lowest-ranked delta features removed.


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Load full dataset
df = pd.read_csv("mfcc_deltas_full.csv")
y = df["label"].map({"orca_call": 1, "no_call": 0})

# Feature columns (skip clip_name and label)
drop_cols = ["delta_mfcc_11", "delta_mfcc_12", "delta_mfcc_13"]
features = [col for col in df.columns if col.startswith("mfcc") or col.startswith("delta_mfcc")]
features = [f for f in features if f not in drop_cols]

X = df[features]

# Train/test split (stratify to preserve class ratio)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate
y_pred = model.predict(X_test_scaled)

print("Confusion Matrix (Final Model):")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["no_call", "orca_call"]))


Confusion Matrix (Final Model):
[[344 138]
 [104 803]]

Classification Report:
              precision    recall  f1-score   support

     no_call       0.77      0.71      0.74       482
   orca_call       0.85      0.89      0.87       907

    accuracy                           0.83      1389
   macro avg       0.81      0.80      0.80      1389
weighted avg       0.82      0.83      0.82      1389

